

---

#word2vec, co-occurrence matrix 를 이용한 레시피 추천

---



In [ ]:
!pip install --upgrade pip
!pip install --upgrade gensim
!pip freeze

In [ ]:
import gensim
import numpy as np
import pandas as pd
import pickle
from numpy import dot
from numpy.linalg import norm
import math

In [ ]:
print( pd.__version__ )

1.3.5


In [ ]:
with open ('/content/drive/MyDrive/Recipe/food_list.p', 'rb') as f:
  food = pickle.load(f)

In [ ]:
def Tokenize(text):
   return text.split(',')
#final_df 불러오기
final_data = pd.read_csv('/content/drive/MyDrive/Recipe/final_recipe_df.csv', index_col = 0)
final_data['ingredients_pre'] = final_data['ingredients_pre'].apply(eval)
title = final_data['name']
lst_a = final_data['ingredients_pre']
#word2vec 불러오기
model = gensim.models.Word2Vec.load('/content/drive/MyDrive/Recipe/wvrecipe')

#tfidf 불러오기
with open ('/content/drive/MyDrive/Recipe/tfidf.p', 'rb') as f:
  tfidf_v = pickle.load(f)

#co-occurrence matrix 불러오기
with open ('/content/drive/MyDrive/Recipe/ingredients_co.p', 'rb') as f:
  data_matrix = pickle.load(f)


#name_matrix 불러오기
with open ('/content/drive/MyDrive/Recipe/name_matrix.p', 'rb') as f:
  name_matrix = pickle.load(f)

In [ ]:
def add_plus(x):
  x += 0.0001
  return x
data_matrix = data_matrix.apply(add_plus)




In [ ]:
data_matrix

,빙어,중국부추,칠면조,캐슈넛,진달래,떡국떡,집간장,은장어,효모,냉이뿌리,...,관자,천일염,참가자미,유자,미꾸라지,샴페인,삼치,명란,까마중,캐비어
빙어,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
중국부추,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
칠면조,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
캐슈넛,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
진달래,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
샴페인,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
삼치,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
명란,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
까마중,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [ ]:
from logging import exception
#재료 추천 개선, 레시피 추천 개선
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
       
def start_adv():
  lst = []
  
  ing = input('먹고 싶은 재료를 선택하세요!(닭고기, 소고기, 돼지고기, 새우 등등) :')


  lst.append(ing)
  cnt = 0
  print('')
  print(list(data_matrix[ing].nlargest(10).index))
  ing_co_matrix = data_matrix[lst[cnt]].copy()
  cnt += 1

       
  while 1:
     
    ing = input('선택한 음식들과 어울리는 재료들 입니다. 이 중에서 선택하세요.!("quit" 입력 시 재료 선택 중단) :')
    if ing == 'quit':
      break
    
      
    lst.append(ing)

    ing_co_matrix *= data_matrix[lst[cnt]].copy()
    ing_co_matrix = ing_co_matrix.apply(math.log)
    rec = list(ing_co_matrix.nlargest(10).index)
    for i in rec:
      if i in lst:
        rec.remove(i)
    print('')      
    print(rec)
    cnt += 1
    ing_co_matrix = ing_co_matrix.apply(math.exp)
    

   
  #레시피 벡터화(word2vec)
  print('')
  print('넣은 재료: {} '.format(lst))
  d = lst
  i_str = ','.join(d)   #가중치 생성
  sentence = [i_str]
  a = tfidf_v.transform(sentence).toarray().flatten()
  w = []
  for i in d:
    kkk = tfidf_v.vocabulary_[i]
    w.append(a[kkk])

  w2 = []  
  for j in range(len(d)):
    w2.append(w[j]/sum(w))

  matrix = np.zeros(shape=(model.wv.vectors.shape[1],))
  for i in range(len(d)):
    matrix += w2[i]*model.wv[d[i]]
    print(w2[i])
  result = matrix/len(d)
  
  #코사인 유사도 다 구하기
  lst2 = []
  for i in range(len(name_matrix)):
    lst2.append(cos_sim(result,name_matrix[i]))

  #코사인 유사도 상위 10개
  n = 30
  for i in np.argsort(lst2)[-n:][::-1]:
    print('')
    print(title[i], lst_a[i], '코사인 유사도: {}'.format(lst2[i]))

In [ ]:
#기본 버전
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
       
def start_basic():
  lst = []
  ing = input('먹고 싶은 재료를 선택하세요!(닭고기, 소고기, 돼지고기, 새우 등등) :')
  lst.append(ing)
  print('')
  print(list(data_matrix[ing].nlargest(10).index))
  while 1:
     
    ing = input('선택한 음식과 어울리는 재료들 입니다. 이 중에서 선택하세요.!("quit" 입력 시 재료 선택 중단) :')

    if ing == 'quit':
      break

    lst.append(ing)
    rec = list(data_matrix[ing].nlargest(10).index)
    for i in rec:
      if i in lst:
        rec.remove(i)
    print('')      
    print(rec)
    
      
   
  #레시피 벡터화(word2vec)
  print('')
  print('넣은 재료: {} '.format(lst))
  d = lst
  matrix = np.zeros(shape=(model.wv.vectors.shape[1],))
  for i in d:
    matrix += model[i]
  result = matrix/len(d)
  
  #코사인 유사도 다 구하기
  lst2 = []
  for i in range(len(name_matrix)):
    lst2.append(cos_sim(result,name_matrix[i]))

  #코사인 유사도 상위 10개
  n = 10
  for i in np.argsort(lst2)[-n:][::-1]:
    print('')
    print(title[i], lst_a[i], '코사인 유사도: {}'.format(lst2[i]))

In [ ]:
start_basic()

In [ ]:
start_adv()

In [ ]:
#주재료에 집중
# 평균(시작)+c(재료(끝)-평균(시작))
#c가 높을수록 주재료에 가까움(0~1사이 값)
def start_main(c = 0.2):
  lst = []
  ing = input('먹고 싶은 재료를 선택하세요!(닭고기, 소고기, 돼지고기, 새우 등등) :')
  lst.append(ing)
  print('')
  print(list(data_matrix[ing].nlargest(10).index))
  while 1:
    # print(lst) 
    ing = input('선택한 음식과 어울리는 재료들 입니다. 이 중에서 선택하세요.!("quit" 입력 시 재료 선택 중단) :')

    if ing == 'quit':
      break

    lst.append(ing)
    rec = list(data_matrix[ing].nlargest(10).index)
    for i in rec:
      if i in lst:
        rec.remove(i)
    print('')      
    print(rec)
    
      
   
  #레시피 벡터화(word2vec)
  print('')
  print('넣은 재료: {} '.format(lst))
  ch = input('넣은 재료들 중에서 주재료를 하나 선택해주세요.(주재료를 입력하면 주재료에 맞는 레시피를 추천할 가능성이 높습니다.) :')
  
  d = lst
  matrix = np.zeros(shape=(model.wv.vectors.shape[1],))
  for i in d:
    matrix += model[i]
    m = matrix/len(d)
  result = m+c*(model[ch]-m)
  
  #코사인 유사도 다 구하기
  lst2 = []
  for i in range(len(name_matrix)):
    lst2.append(cos_sim(result,name_matrix[i]))

  #코사인 유사도 상위 10개
  n = 20
  for i in np.argsort(lst2)[-n:][::-1]:
    print('')
    print(i, title[i], lst_a[i], '코사인 유사도: {}'.format(lst2[i]))

In [ ]:
start_main()

In [ ]:
# num = 62660 
# l = []
# for i in range(len(title_matrix)):
#   ko = cos_sim(title_matrix[num], title_matrix[i])
#   if 1> ko > 0.93:
#     print(title[i], ko)


시래기 고등어조림 만드는 법 0.9319728100126669
고등어조림  0.9442194876305458
고등어무조림 만드는 법 비린내 없이 칼칼하게 ~  0.9391018831454567
초보도 할 수 있는 고등어조림 만들기 0.9329829590366568
고등어조림♡ 0.930766709152915
간단요리 동태매운탕 0.9339478866327982
고등어김치찜 밥도둑이 따로 없어요 (비린내 완벽 제거) 0.9468501492896806
